In [1]:
!pip install sqlalchemy==1.3.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 51.7 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.3.9-cp37-cp37m-linux_x86_64.whl size=1159121 sha256=629da0f74fdd77e021880337aea822666e21ac23e5e0a2ba389445dae65d266d
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/03/71/13/010faf12246f72dc76b4150e6e599d13a85b4435e06fb9e51f
Successfully built sqlalchemy
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.24
    Uninstalling SQLAlchemy-1.3.24:
      Successfully uninstalled SQLAlchemy-1.3.24


In [2]:
%load_ext sql

In [3]:
import csv, sqlite3

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

In [4]:
!pip install -q pandas==1.1.5

In [5]:
%sql sqlite:///my_data1.db

'Connected: @my_data1.db'

In [6]:
import pandas as pd
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv")
df.to_sql("SPACEXTBL", con, if_exists='replace', index=False,method="multi")

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [7]:
# get dataset into pandas dataframe
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01"
df = pd.read_csv(url)
# remove spaces in columns name
df.columns = df.columns.str.replace(' ','_')

# convert timestamp strings to date and time format
df['Date'] = pd.to_datetime(df['Date'], errors="coerce", format="%d-%m-%Y")

print(df.dtypes)
print(df.head())

Date                 datetime64[ns]
Time_(UTC)                   object
Booster_Version              object
Launch_Site                  object
Payload                      object
PAYLOAD_MASS__KG_           float64
Orbit                        object
Customer                     object
Mission_Outcome              object
Landing_Outcome              object
dtype: object
  Date Time_(UTC) Booster_Version  Launch_Site  \
0  NaT   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  NaT   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  NaT    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  NaT    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  NaT   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats, barrel of...                0.0   
2                              Dragon demo flight C2              525.0   
3                        

In [8]:
# create sqlite table and upload data into it
conn = sqlite3.connect(':memory:')  # in memory database
df.to_sql(name="spacexdata", con=conn, if_exists="replace")

q = pd.read_sql('select * from spacexdata', conn)
q

,index,Date,Time_(UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,0,None,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Success,Failure (parachute)
1,1,None,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0.0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,2,None,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525.0,LEO (ISS),NASA (COTS),Success,No attempt
3,3,None,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),Success,No attempt
4,4,None,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),Success,No attempt
...,...,...,...,...,...,...,...,...,...,...,...
994,994,None,None,None,None,None,NaN,None,None,None,None
995,995,None,None,None,None,None,NaN,None,None,None,None
996,996,None,None,None,None,None,NaN,None,None,None,None
997,997,None,None,None,None,None,NaN,None,None,None,None


In [10]:
# Task 1 Display the names of the unique launch sites in the space mission#
q = pd.read_sql('select distinct Launch_Site from spacexdata', conn)
q

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40
4,None


In [12]:
# Task 2 Display 5 records where launch sites begin with the string 'KSC#
q = pd.read_sql("select * from spacexdata where Launch_Site like 'KSC%' limit 5", conn)
q

,index,Date,Time_(UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,29,None,14:39:00,F9 FT B1031.1,KSC LC-39A,SpaceX CRS-10,2490.0,LEO (ISS),NASA (CRS),Success,Success (ground pad)
1,30,None,6:00:00,F9 FT B1030,KSC LC-39A,EchoStar 23,5600.0,GTO,EchoStar,Success,No attempt
2,31,None,22:27:00,F9 FT B1021.2,KSC LC-39A,SES-10,5300.0,GTO,SES,Success,Success (drone ship)
3,32,None,11:15:00,F9 FT B1032.1,KSC LC-39A,NROL-76,5300.0,LEO,NRO,Success,Success (ground pad)
4,33,None,23:21:00,F9 FT B1034,KSC LC-39A,Inmarsat-5 F4,6070.0,GTO,Inmarsat,Success,No attempt


In [13]:
# Task 3 Display the total payload mass carried by boosters launched by NASA (CRS)#

q = pd.read_sql("select sum(PAYLOAD_MASS__KG_) from spacexdata where Customer='NASA (CRS)'", conn)
q

,sum(PAYLOAD_MASS__KG_)
0,45596.0


In [14]:
# Task 4 Display average payload mass carried by booster version F9 v1.1¶#
q = pd.read_sql("select avg(PAYLOAD_MASS__KG_) from spacexdata where Booster_Version='F9 v1.1'", conn)
q


,avg(PAYLOAD_MASS__KG_)
0,2928.4


In [16]:
#Task 5 List the date where the succesful landing outcome in drone ship was acheived.#
q = pd.read_sql("select min(Date) from spacexdata where Landing_Outcome='Success (drone ship)'", conn)
q

,min(Date)
0,None


In [18]:
#Task 6 List the names of the boosters which have success in ground pad and have payload mass greater than 4000 but less than 6000#
q = pd.read_sql("select distinct Booster_Version from spacexdata where Landing_Outcome='Success (ground pad)' and PAYLOAD_MASS__KG_ between 4000 and 6000", conn)
q

,Booster_Version
0,F9 FT B1032.1
1,F9 B4 B1040.1
2,F9 B4 B1043.1


In [22]:
# Task 7 List the total number of successful and failure mission outcomes#

q = pd.read_sql("select substr(Mission_Outcome,1,7) as Mission_Outcome, count(*) from spacexdata  group by 1", conn)
q

,Mission_Outcome,count(*)
0,None,898
1,Failure,1
2,Success,100


In [21]:
#Task 8 List the names of the booster_versions which have carried the maximum payload mass. Use a subquery#

q = pd.read_sql("select distinct Booster_Version from spacexdata where PAYLOAD_MASS__KG_ = (select max(PAYLOAD_MASS__KG_) from spacexdata)", conn)
q

,Booster_Version
0,F9 B5 B1048.4
1,F9 B5 B1049.4
2,F9 B5 B1051.3
3,F9 B5 B1056.4
4,F9 B5 B1048.5
5,F9 B5 B1051.4
6,F9 B5 B1049.5
7,F9 B5 B1060.2
8,F9 B5 B1058.3
9,F9 B5 B1051.6


In [63]:
#Task 9 List the records which will display the month names, succesful landing_outcomes in ground pad ,booster versions, launch_site for the months in year 2017¶#

import sqlite3

def get_records():
    conn = sqlite3.connect('your_import sqlite3')

def get_records():
    conn = sqlite3.connect('your_Spacexdataset.db')  # Replace 'your_database.db' with the actual name of your SQLite database file
    c = conn.cursor()

    c.execute("SELECT substr(Date, 4, 2) AS Month, successful_landings, booster_version, launch_site FROM your_table WHERE substr(Date, 7, 4) = '2017'")

    records = c.fetchall()

    conn.close()

    return records

# Usage example:

    results = get_records()
    for record in results:
    print(f"Month: {record[0]}, Successful Landings: {record[1]}, Booster Version: {record[2]}, Launch Site: {record[3]}")



    c = conn.cursor()

    c.execute("SELECT substr(Date, 4, 2) AS Month, successful_landings, booster_version, launch_site FROM your_table WHERE substr(Date, 7, 4) = '2017'")

    records = c.fetchall()

    conn.close()

    return records

    # Usage example:
    results = get_records()
    for record in results:
    print(f"Month: {record[0]}, Successful Landings: {record[1]}, Booster Version: {record[2]}, Launch Site: {record[3]}"
    #Please make sure to replace 'Spacexdataset.db' with the actual name of your SQLite database file, and 'your_table' with the name of the table in which your data is stored.#




import requests

def get_launch_records(year):
    url = f"https://api.spacexdata.com/v2/launches?launch_year={year}"
    response = requests.get(url)
    if response.status_code == 200:
        launches = response.json()
        records = []
        for launch in launches:
            month = launch['launch_date_local'][5:7]
            if month not in records and launch['launch_success'] and launch['landing_type'] == 'Land' and launch['landing_vehicle'] == 'Ground pad':
                records.append(month)

        return records
    else:
        print(f"Error: {response.status_code}")
        return []

def get_launch_info(month):
    url = f"https://api.spacexdata.com/v2/launches?launch_year=2017&launch_success=true&land_success=true&landing_type=Land&landing_vehicle=Ground pad"
    response = requests.get(url)
    if response.status_code == 200:
        launches = response.json()
        for launch in launches:
            if launch['launch_date_local'][5:7] == month:
                print("Month:", month)
                print("Successful Landing Outcomes:", launch['launch_success'])
                print("Booster Version:", launch['rocket']['second_stage']['payloads'][0]['reused'])
                print("Launch Site:", launch['launch_site']['site_name'])
                print("------------------------")
    else:
        print(f"Error: {response.status_code}")

        year = 2017
        records = get_launch_records(year)

        for month in records:
            get_launch_info(month)



IndentationError: expected an indented block (372077289.py, line 24)

In [42]:
#Task10#

import requests

def get_landing_outcomes(start_date, end_date):
    url = f"https://api.spacexdata.com/v2/launches?start={start_date}&end={end_date}"
    response = requests.get(url)
    if response.status_code == 200:
        launches = response.json()
        outcomes = {}
        for launch in launches:
            landing_outcome = launch['landing_type']
            if landing_outcome not in outcomes:
                outcomes[landing_outcome] = 1
            else:
                outcomes[landing_outcome] += 1

        sorted_outcomes = sorted(outcomes.items(), key=lambda x: x[1], reverse=True)
        return sorted_outcomes
    else:
        print(f"Error: {response.status_code}")
        return []
    start_date = "2010-06-04"
    end_date = "2017-03-20"

    landing_outcomes = get_landing_outcomes(start_date, end_date)

    print("Landing Outcome Rankings:")
    for outcome, count in landing_outcomes:
        print(f"{outcome}: {count}")

In [59]:
#Task 10 #

import pandas as pd
import sqlite3

conn = sqlite3.connect('your_database.db')  # Replace 'your_database.db' with the actual name of your SQLite database file

query = """
    SELECT Landing__Outcome, COUNT(*) AS Count
    FROM spacexdata
    WHERE Date BETWEEN '2011-06-04' AND '2017-03-20'
    GROUP BY Landing__Outcome
    ORDER BY Count DESC
"""

df = pd.read_sql(query, conn)

conn.close()

print(df)
#Make sure to replace 'your_database.db' with the actual name of your SQLite database file. After executing the code, the resulting DataFrame (df) will contain the landing outcomes and their corresponding counts, sorted in descending order by count.#










DatabaseError: Execution failed on sql '
    SELECT Landing__Outcome, COUNT(*) AS Count
    FROM spacexdata
    WHERE Date BETWEEN '2011-06-04' AND '2017-03-20'
    GROUP BY Landing__Outcome
    ORDER BY Count DESC
': no such table: spacexdata